In [117]:
# default_exp digdata

# digdata

> 探索数据.

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
#export 
from bs4 import BeautifulSoup
from parser import * #regex_parser
import re
import json
import time
import logging
import datetime
import requests
import pprint 

In [58]:
#export 
#url = "https://3g.dxy.cn/newh5/view/pneumonia"
url = "https://ncov.dxy.cn/ncovh5/view/pneumonia?from=singlemessage&isappinstalled=0"
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'
}

In [59]:
#export 
session = requests.session()
session.headers.update(headers)

In [60]:
#export 
r = session.get(url)

In [61]:
#export 
#pprint.pprint(r.text)

In [62]:
#export 
soup = BeautifulSoup(r.content, 'lxml')

In [10]:
#export 
def getdata():
    return 2

In [63]:
#export 
overall_information = re.search(r'\{("id".*?)\}', str(soup.find('script', attrs={'id': 'getStatisticsService'})))
province_information = re.search(r'\[(.*?)\]', str(soup.find('script', attrs={'id': 'getListByCountryTypeService1'})))
area_information = re.search(r'\[(.*)\]', str(soup.find('script', attrs={'id': 'getAreaStat'})))
news = re.search(r'\[(.*?)\]', str(soup.find('script', attrs={'id': 'getTimelineService'})))

In [106]:
#pprint.pprint(overall_information.string)

In [104]:
#overall_information.group(0)

In [107]:
#jsall = json.loads(overall_information.group(0))

In [21]:
def overall_parser(overall_information):
    overall_information = json.loads(overall_information.group(0))
    overall_information.pop('id')
    overall_information.pop('createTime')
    overall_information.pop('modifyTime')
    overall_information.pop('imgUrl')
    overall_information.pop('deleted')
    overall_information['countRemark'] = overall_information['countRemark'].replace(' 疑似', '，疑似').replace(' 治愈', '，治愈').replace(' 死亡', '，死亡').replace(' ', '')
    if not self.db.find_one(collection='DXYOverall', data=overall_information):
        overall_information['updateTime'] = self.crawl_timestamp
        overall_information = regex_parser(content=overall_information, key='countRemark')

In [108]:
#overall_parser(overall_information)

In [109]:
#province_information.string

In [112]:
#provinces = json.loads(province_information.group(0))

In [110]:
#provinces

In [113]:
#for province in provinces:
#    print(province['id'],'\t',province['provinceShortName'],'\t',province['tags'])

In [40]:
def province_parser(province_information):
    provinces = json.loads(province_information.group(0))
    crawl_timestamp = ""
    for province in provinces:

        province.pop('id')
        province['comment'] = province['comment'].replace(' ', '')
        province['crawlTime'] = crawl_timestamp
        #province['country'] = country_type.get(province['countryType'])
        province['tags'] = province['tags'].replace(' ', '')
        province = regex_parser(content=province, key='tags')

In [114]:
#province_parser(province_information)

In [115]:
#area_information.string

In [72]:
area = json.loads(area_information.group(0))

In [116]:
print("省份\t确诊\t疑似\t治愈\t死亡")
for a in area:
    print(a['provinceName'],'\t',a['confirmedCount'],'\t',a['suspectedCount'],'\t',a['curedCount'],'\t',a['deadCount'])

省份	确诊	疑似	治愈	死亡
湖北省 	 22112 	 0 	 856 	 618
广东省 	 1034 	 0 	 88 	 1
浙江省 	 1006 	 0 	 123 	 0
河南省 	 914 	 0 	 80 	 3
湖南省 	 772 	 0 	 112 	 0
安徽省 	 665 	 0 	 47 	 0
江西省 	 661 	 0 	 45 	 0
重庆市 	 415 	 0 	 31 	 2
江苏省 	 408 	 0 	 41 	 0
山东省 	 386 	 0 	 37 	 0
四川省 	 344 	 0 	 42 	 1
北京市 	 297 	 0 	 33 	 1
上海市 	 277 	 166 	 30 	 1
黑龙江省 	 277 	 0 	 12 	 3
福建省 	 224 	 0 	 16 	 0
陕西省 	 184 	 0 	 17 	 0
广西壮族自治区 	 172 	 0 	 17 	 0
河北省 	 172 	 0 	 22 	 1
云南省 	 136 	 0 	 13 	 0
海南省 	 117 	 0 	 10 	 2
辽宁省 	 96 	 0 	 7 	 0
山西省 	 96 	 0 	 15 	 0
贵州省 	 81 	 0 	 6 	 1
天津市 	 81 	 0 	 2 	 1
甘肃省 	 67 	 0 	 9 	 0
吉林省 	 65 	 0 	 4 	 1
内蒙古自治区 	 50 	 0 	 5 	 0
宁夏回族自治区 	 43 	 0 	 5 	 0
新疆维吾尔自治区 	 39 	 0 	 0 	 0
香港 	 25 	 0 	 0 	 1
青海省 	 18 	 0 	 3 	 0
台湾 	 16 	 0 	 1 	 0
澳门 	 10 	 0 	 1 	 0
西藏自治区 	 1 	 0 	 0 	 0


In [ ]:
for p in area:

In [94]:
cities = area[0]['cities']

In [103]:
print("城市\t确诊\t疑似\t治愈\t死亡")
for p in area:
    cities = p['cities']
    print("===================================")
    print(p['provinceName'],'\t',p['confirmedCount'],'\t',p['suspectedCount'],'\t',p['curedCount'],'\t',p['deadCount'])
    print("-----------------------------------")
    for c in cities:
        print(c['cityName'],'\t',c['confirmedCount'],'\t',c['suspectedCount'],'\t',a['curedCount'],'\t',a['deadCount'])

城市	确诊	疑似	治愈	死亡
湖北省 	 22112 	 0 	 856 	 618
-----------------------------------
武汉 	 11618 	 0 	 0 	 0
孝感 	 2141 	 0 	 0 	 0
黄冈 	 1897 	 0 	 0 	 0
随州 	 915 	 0 	 0 	 0
荆州 	 885 	 0 	 0 	 0
襄阳 	 838 	 0 	 0 	 0
黄石 	 635 	 0 	 0 	 0
宜昌 	 610 	 0 	 0 	 0
荆门 	 553 	 0 	 0 	 0
鄂州 	 471 	 0 	 0 	 0
咸宁 	 443 	 0 	 0 	 0
十堰 	 395 	 0 	 0 	 0
仙桃 	 307 	 0 	 0 	 0
天门 	 163 	 0 	 0 	 0
恩施州 	 157 	 0 	 0 	 0
潜江 	 74 	 0 	 0 	 0
神农架林区 	 10 	 0 	 0 	 0
广东省 	 1034 	 0 	 88 	 1
-----------------------------------
深圳 	 339 	 0 	 0 	 0
广州 	 285 	 0 	 0 	 0
珠海 	 76 	 0 	 0 	 0
佛山 	 59 	 0 	 0 	 0
东莞 	 53 	 0 	 0 	 0
中山 	 46 	 0 	 0 	 0
惠州 	 39 	 0 	 0 	 0
汕头 	 22 	 0 	 0 	 0
湛江 	 20 	 0 	 0 	 0
肇庆 	 14 	 0 	 0 	 0
江门 	 14 	 0 	 0 	 0
阳江 	 13 	 0 	 0 	 0
梅州 	 12 	 0 	 0 	 0
清远 	 10 	 0 	 0 	 0
揭阳 	 7 	 0 	 0 	 0
韶关 	 6 	 0 	 0 	 0
茂名 	 6 	 0 	 0 	 0
汕尾 	 5 	 0 	 0 	 0
潮州 	 5 	 0 	 0 	 0
河源 	 3 	 0 	 0 	 0
浙江省 	 1006 	 0 	 123 	 0
-----------------------------------
温州 	 421 	 0 	 0 	 0
杭州 	 156 	 0 	 0 	 0

In [80]:
news.string

'<script id="getTimelineService">try { window.getTimelineService = [{"id":2732,"pubDate":1581078074000,"pubDateStr":"6分钟前","title":"深圳所有小区封闭管理，确诊病例楼14天硬隔离","summary":"7日，深圳发布《关于建立健全社区联防联控机制的若干措施》，明确了“五个100%”、“十个一律”、“四个重点”等防控举措：社区小区100%实行围合封闭式管理；近14天有疫情重点地区旅居史的人员100%居家隔离等。","body":"【#深圳所有小区封闭管理#，#深圳确诊病例楼14天硬隔离#】7日，深圳发布《关于建立健全社区联防联控机制的若干措施》，明确了“五个100%”、“十个一律”、“四个重点”等防控举措：社区小区100%实行围合封闭式管理；近14天有疫情重点地区旅居史的人员100%居家隔离；存在与病例密切接触史的人员100%实...全文： http://m.weibo.cn/2803301701/4469427740938572 \u200b","infoSource":"人民日报","sourceUrl":"http://m.weibo.cn/2803301701/4469427740938572","provinceId":"44","provinceName":"广东省","createTime":1581078379000,"modifyTime":1581078379000,"entryWay":1,"adoptType":1,"infoType":2,"dataInfoState":0,"dataInfoOperator":"","dataInfoTime":1581078379000},{"id":2722,"pubDate":1581074305000,"pubDateStr":"1小时前","title":"钱七虎院士为武汉捐款650万","summary":"疫情当前，中国工程院院士钱七虎分两次向武汉捐款650万元。年过八旬的钱老，是我国著名防护工程专家，现任军委科技委顾问、陆军工程大学教授。去年，他就曾把获得2018年度国家最高科技奖的奖金800万元全部捐赠，资助贫困学生上学。\\n","body":"#万众一心抗

In [11]:
from nbdev.export import *
notebook2script()

Converted 00_getdata.ipynb.
Converted index.ipynb.
